In [1]:
#! pip install autowoe

# Imports

In [3]:
import pandas as pd
import numpy as np
from autowoe import AutoWoE
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Load data & prepare

In [4]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
submission_df = pd.read_csv('submission.csv')

In [5]:
"""def create_col_with_min_freq(data, col, min_freq = 10):
    # replace rare values (less than min_freq rows) in feature by RARE_VALUE
    data[col + '_fixed'] = data[col].astype(str)
    data.loc[data[col + '_fixed'].value_counts()[data[col + '_fixed']].values < min_freq, col + '_fixed'] = "RARE_VALUE"
    data.replace({'nan': np.nan}, inplace = True)
    
def create_gr_feats(data):
    # create aggregation feats for numeric features based on categorical ones
    for cat_col in ['feat_100_1', 'feat_100_2', 'feat_100_3',
                   'feat_100_4', 'feat_100_5',]:
        create_col_with_min_freq(data, cat_col, 15)
        for num_col in ['C_1', 'B_1',]:
            for n, f in [('mean', np.mean), ('min', np.nanmin), ('max', np.nanmax)]:
                data['FIXED_' + n + '_' + num_col + '_by_' + cat_col] = data.groupby(cat_col + '_fixed')[num_col].transform(f)
                
    # create features with counts
    for col in ['feat_100_1', 'feat_100_2', 'feat_100_3',
                   'feat_100_4', 'feat_100_5']:
        data[col + '_cnt'] = data[col].map(data[col].value_counts(dropna = False))
        

all_df = pd.concat([train_data, test_data]).reset_index(drop = True)
create_gr_feats(all_df)
train_data, test_data = all_df[:len(train_data)], all_df[len(train_data):]"""

'def create_col_with_min_freq(data, col, min_freq = 10):\n    # replace rare values (less than min_freq rows) in feature by RARE_VALUE\n    data[col + \'_fixed\'] = data[col].astype(str)\n    data.loc[data[col + \'_fixed\'].value_counts()[data[col + \'_fixed\']].values < min_freq, col + \'_fixed\'] = "RARE_VALUE"\n    data.replace({\'nan\': np.nan}, inplace = True)\n    \ndef create_gr_feats(data):\n    # create aggregation feats for numeric features based on categorical ones\n    for cat_col in [\'feat_100_1\', \'feat_100_2\', \'feat_100_3\',\n                   \'feat_100_4\', \'feat_100_5\',]:\n        create_col_with_min_freq(data, cat_col, 15)\n        for num_col in [\'C_1\', \'B_1\',]:\n            for n, f in [(\'mean\', np.mean), (\'min\', np.nanmin), (\'max\', np.nanmax)]:\n                data[\'FIXED_\' + n + \'_\' + num_col + \'_by_\' + cat_col] = data.groupby(cat_col + \'_fixed\')[num_col].transform(f)\n                \n    # create features with counts\n    for col in [

In [6]:
train_data.head()

,id,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,...,B_56,B_57,B_58,B_59,feat_100_1,feat_100_2,feat_100_3,feat_100_4,feat_100_5,target
0,0,-1.7790,-0.6060,0.1906,0.3062,-1.5210,-0.2082,0.5510,0.1784,-0.5297,...,-1.025402,0.231713,0.073569,-0.359106,0,2,1,2,3,0
1,1,-0.1853,-0.1236,0.7319,-0.4460,-0.0055,0.2396,-0.5211,0.7877,-0.3647,...,1.448978,-1.894126,0.141569,-0.899142,0,0,0,0,2,0
2,2,1.1940,0.2155,0.6262,1.7200,0.0688,1.0690,1.0990,0.9081,0.6513,...,-0.521487,-1.272666,-0.668533,-0.993069,1,2,0,2,3,0
3,3,0.5445,0.1486,-1.0790,-0.7789,0.0671,-0.5055,-0.9356,0.4150,-0.2520,...,1.371463,-0.343640,0.289540,-0.714268,0,2,0,2,2,0
4,4,-1.0880,-0.4271,1.1520,0.1212,0.5306,-2.2490,0.7976,0.0174,-0.0495,...,0.041749,0.187665,0.841361,-0.383784,0,2,0,2,2,0


In [8]:
train_data.drop('id', axis=1, inplace=True)
test_data.drop('id', axis=1, inplace=True)

In [9]:
train_df, valid_df = train_test_split(
    train_data,
    stratify=train_data['target'],
    shuffle=True,
    random_state=100
)

# Training

In [17]:
autowoe = AutoWoE(verbose=0,
                monotonic=False,
                select_type = 100,
                 vif_th=20.,
                 th_const=32,
                 force_single_split=True,
                 oof_woe=True,
                 n_folds=5,
                 n_jobs=4,
                 regularized_refit=True)

In [18]:
max_bin_count = {col: 7 for col in train_df.columns}

In [19]:
autowoe.fit(
    train_df, 
    target_name="target",
    #max_bin_count=max_bin_count,
)

In [20]:
pred = autowoe.predict_proba(valid_df)
roc_auc_score(valid_df['target'], pred)

0.8289010463103793

# Make submission

In [21]:
pred = autowoe.predict_proba(test_data)

In [22]:
submission_df['pred'] = pred

In [23]:
submission_df.to_csv('baseline_submission.csv', index=False)